### Data ###

In [1]:
import xray

In [2]:
data = xray.open_dataset('tracks.20130125.nc', decode_cf=False)

In [3]:
# Dataset Information
print data

<xarray.Dataset>
Dimensions:  (Nobs: 2590938)
Coordinates:
  * Nobs     (Nobs) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 ...
Data variables:
    A        (Nobs) float32 1.24564 2.64604 3.40119 1.45836 2.16156 4.87579 ...
    track    (Nobs) int32 1 1 1 1 2 2 2 2 3 3 3 3 4 4 4 4 5 5 5 5 6 6 6 6 7 ...
    lon      (Nobs) float32 636.74 635.99 635.82 635.845 560.642 561.447 ...
    j1       (Nobs) int32 2448910 2448917 2448924 2448931 2448910 2448917 ...
    n        (Nobs) int32 1 2 3 4 1 2 3 4 1 2 3 4 1 2 3 4 1 2 3 4 1 2 3 4 1 ...
    cyc      (Nobs) int32 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 -1 ...
    U        (Nobs) float32 5.97993 8.17373 8.28628 5.20364 4.81408 6.59872 ...
    lat      (Nobs) float32 -65.0011 -65.1376 -65.2457 -65.2933 -62.5424 ...
    L        (Nobs) float32 44.464 36.7024 45.4221 43.022 63.0697 51.1345 ...
Attributes:
    data_start_date: Oct 14, 1992
    data_end_date: Apr  4, 2012
    title: Mesoscale Eddies in Altimeter Observations of

### Multi-Index ###

In [4]:
import numpy as np
import pandas as pd

In [5]:
ds = data.to_dataframe()

In [6]:
ds.tail(4)

,A,track,lon,j1,n,cyc,U,lat,L
Nobs,,,,,,,,,
2590934,3.06445,215184,336.242004,2456001,1,1,7.21748,62.275398,54.620499
2590935,2.03734,215184,336.033997,2456008,2,1,5.87109,62.011700,53.794102
2590936,1.75523,215184,335.938995,2456015,3,1,6.72342,62.359200,30.240000
2590937,1.23239,215184,335.752991,2456022,4,1,6.36023,62.145901,37.190102


In [7]:
# Dimension Setting
# All
# M = max(ds.track)
# N = len(ds.n)

In [8]:
# Dimension Setting
# Test
M = 3000
N = 41320

In [9]:
ds = ds.head(N)

In [10]:
ds.tail(4)

,A,track,lon,j1,n,cyc,U,lat,L
Nobs,,,,,,,,,
41316,4.50767,3000,585.187988,2449043,1,-1,43.786400,-1.012250,129.462006
41317,5.60816,3000,577.690979,2449050,2,-1,91.454697,-0.200525,107.625999
41318,3.18113,3000,577.781982,2449057,3,-1,42.783001,-0.156585,190.341003
41319,2.38252,3000,574.455994,2449064,4,-1,23.117800,-0.401030,156.994995


In [11]:
track = np.asarray(ds.track)
n = np.asarray(ds.n)

In [12]:
# Multi-Index Construction
arrays = [track, n]
tuples = list(zip(*arrays))
multi_index = pd.MultiIndex.from_tuples(tuples, names=['Eddy ID', 'Point ID'])

In [13]:
j1 = np.asarray(ds.j1)
cyc = np.asarray(ds.cyc)
lon = np.asarray(ds.lon)
lat = np.asarray(ds.lat)
A = np.asarray(ds.A)
L = np.asarray(ds.L)
U = np.asarray(ds.U)

In [14]:
var = np.zeros((N, 7))

In [15]:
for i in range(N):
    var[i][0] = j1[i]
    var[i][1] = cyc[i]
    var[i][2] = lon[i]
    var[i][3] = lat[i]
    var[i][4] = A[i]
    var[i][5] = L[i]
    var[i][6] = U[i]

In [16]:
df = pd.DataFrame(var, index=multi_index)
df.columns = ['j1', 'cyc', 'lon', 'lat', 'A', 'L', 'U']

In [17]:
df.tail(4)

j1  cyc         lon       lat        A           L  \
Eddy ID Point ID                                                              
3000    1         2449043.0 -1.0  585.187988 -1.012250  4.50767  129.462006   
        2         2449050.0 -1.0  577.690979 -0.200525  5.60816  107.625999   
        3         2449057.0 -1.0  577.781982 -0.156585  3.18113  190.341003   
        4         2449064.0 -1.0  574.455994 -0.401030  2.38252  156.994995   

                          U  
Eddy ID Point ID             
3000    1         43.786400  
        2         91.454697  
        3         42.783001  
        4         23.117800

### Time ###

In [18]:
import datetime, jdcal

In [19]:
def jday_to_datetime(jday, refday=0):
    y, m, d, f = jdcal.jd2gcal(jday, refday)
    h = int(f*24)
    return pd.to_datetime(datetime.datetime(y, m, d, h))

In [20]:
pd_date = df.j1.apply(jday_to_datetime)

In [21]:
df.j1 = pd_date
df = df.rename(columns = {'j1': 'date'})

In [22]:
df.tail(4)

date  cyc         lon       lat        A  \
Eddy ID Point ID                                                           
3000    1        1993-02-24 12:00:00 -1.0  585.187988 -1.012250  4.50767   
        2        1993-03-03 12:00:00 -1.0  577.690979 -0.200525  5.60816   
        3        1993-03-10 12:00:00 -1.0  577.781982 -0.156585  3.18113   
        4        1993-03-17 12:00:00 -1.0  574.455994 -0.401030  2.38252   

                           L          U  
Eddy ID Point ID                         
3000    1         129.462006  43.786400  
        2         107.625999  91.454697  
        3         190.341003  42.783001  
        4         156.994995  23.117800

### Longitude ###

In [23]:
# Longitude Range Setting
# From -180 to 180
lon_fix_01 = df.where(df.lon <= 540).lon - 360
lon_fix_02 = df.where(df.lon > 540).lon - 720
lon_fix_A = lon_fix_01
lon_fix_A = lon_fix_A.fillna(lon_fix_02)
df.lon = lon_fix_A

In [24]:
# Longitude Range Setting
# From 0 to 360
lon_fix_03 = df.where(df.lon < 0).lon + 360
lon_fix_04 = df.where(df.lon >= 0).lon
lon_fix_B = lon_fix_03
lon_fix_B = lon_fix_B.fillna(lon_fix_04)
df.lon = lon_fix_B

In [25]:
df.tail(4)

date  cyc         lon       lat        A  \
Eddy ID Point ID                                                           
3000    1        1993-02-24 12:00:00 -1.0  225.187988 -1.012250  4.50767   
        2        1993-03-03 12:00:00 -1.0  217.690979 -0.200525  5.60816   
        3        1993-03-10 12:00:00 -1.0  217.781982 -0.156585  3.18113   
        4        1993-03-17 12:00:00 -1.0  214.455994 -0.401030  2.38252   

                           L          U  
Eddy ID Point ID                         
3000    1         129.462006  43.786400  
        2         107.625999  91.454697  
        3         190.341003  42.783001  
        4         156.994995  23.117800

### Point ###

In [26]:
count = ds.track
count = count.value_counts(normalize=False, sort=True, ascending=True, bins=None, dropna=False).reindex(range(1, M+1))
count = pd.DataFrame(count)
count.columns = ['point']

In [27]:
count.tail(4)

,point
2997,4
2998,4
2999,4
3000,4


In [28]:
lon = np.asarray(df.lon)
lat = np.asarray(df.lat)
point = np.asarray(count.point)

In [29]:
# Origin Coordinates
lon_o = np.zeros(M)
lat_o = np.zeros(M)

In [30]:
# Termination Coordinates
lon_t = np.zeros(M)
lat_t = np.zeros(M)

In [31]:
c = 0
i = 0

while i < N:
    c = int(c) + 1
    lon_o[c-1] = lon[i]
    lat_o[c-1] = lat[i]
    i = i + int(point[c-1])
    lon_t[c-1] = lon[i-1]
    lat_t[c-1] = lat[i-1]

In [32]:
lon_o = pd.DataFrame(lon_o)
lat_o = pd.DataFrame(lat_o)
lon_t = pd.DataFrame(lon_t)
lat_t = pd.DataFrame(lat_t)

In [33]:
# Index Construction
index = np.zeros(M)

In [34]:
for i in range(M):
    index[i] = i+1

In [35]:
index = pd.DataFrame(index)
index = index.astype(int)

In [36]:
# Start Centers
sc = pd.concat([index, lon_o, lat_o], axis=1)
sc.columns = ['Eddy ID', 'lon', 'lat']
sc = sc.set_index('Eddy ID')

In [37]:
sc.tail(4)

,lon,lat
Eddy ID,,
2997,190.353027,-38.660000
2998,34.248993,-31.216299
2999,14.755005,-30.001801
3000,225.187988,-1.012250


In [38]:
# End Centers
ec = pd.concat([index, lon_t, lat_t], axis=1)
ec.columns = ['Eddy ID', 'lon', 'lat']
ec = ec.set_index('Eddy ID')

In [39]:
ec.tail(4)

,lon,lat
Eddy ID,,
2997,189.864990,-38.80320
2998,32.817993,-31.36990
2999,15.549011,-30.23790
3000,214.455994,-0.40103


### LineString ###

In [40]:
ls = np.zeros((N, 2))

In [41]:
for i in range(N):
    ls[i][0] = lon[i]
    ls[i][1] = lat[i]

In [42]:
ls = pd.DataFrame(ls, index=multi_index, columns=['lon', 'lat'])

In [43]:
ls.tail(4)

lon       lat
Eddy ID Point ID                      
3000    1         225.187988 -1.012250
        2         217.690979 -0.200525
        3         217.781982 -0.156585
        4         214.455994 -0.401030

### Polygon ###

In [44]:
from numpy import cos, pi, sin

In [45]:
L = np.asarray(df.L)

In [46]:
L_o = np.zeros(M)
L_t = np.zeros(M)

In [47]:
c = 0
i = 0

while i < N:
    c = int(c) + 1
    L_o[c-1] = L[i]
    i = i + int(point[c-1])
    L_t[c-1] = L[i-1]

In [48]:
L_o = pd.DataFrame(L_o)
L_t = pd.DataFrame(L_t)

In [49]:
# Origin Circles
oc = pd.concat([lon_o, lat_o, L_o], axis=1)
oc.columns = ['lon', 'lat', 'L']

In [50]:
oc.tail(4)

,lon,lat,L
2996,190.353027,-38.660000,74.715797
2997,34.248993,-31.216299,102.660004
2998,14.755005,-30.001801,75.933502
2999,225.187988,-1.012250,129.462006


In [51]:
# Termination Circles
tc = pd.concat([lon_t, lat_t, L_t], axis=1)
tc.columns = ['lon', 'lat', 'L']

In [52]:
tc.tail(4)

,lon,lat,L
2996,189.864990,-38.80320,66.479401
2997,32.817993,-31.36990,68.051003
2998,15.549011,-30.23790,75.832100
2999,214.455994,-0.40103,156.994995


In [53]:
# Circle Center Number
C = M

In [54]:
# Circle Arc Number
A = C*17

In [55]:
center = np.zeros(A)
arc = np.zeros(A)

In [56]:
c = 1
i = 0
j = 1

while i < A:
    while c <= 17:
        center[i] = j
        i = i+1
        c = c+1
    j = j+1
    c = 1

In [57]:
center = center.astype(int)

In [58]:
c = 1
i = 0
j = 1

while i < A:
    while c <= 17:
        arc[i] = j
        i = i+1
        j = j+1
        c = c+1
    j = 1
    c = 1

In [59]:
arc = arc.astype(int)

In [60]:
# Multi-Index Construction
arrays = [center, arc]
tuples = list(zip(*arrays))
multi_index = pd.MultiIndex.from_tuples(tuples, names=['center', 'arc'])

In [61]:
# Earth Radius in Kilometers
R = 6371

__Origins__

In [62]:
lon = np.asarray(oc.lon)
lat = np.asarray(oc.lat)
L = np.asarray(oc.L)

In [63]:
theta = np.zeros(C)
x = np.zeros(C)
y = np.zeros(C)
r = np.zeros(C)

In [64]:
for i in range(C):
    theta[i] = lat[i]*(pi/180)
    r[i] = R*cos(theta[i])
    x[i] = (L[i]/r[i])*(180/pi)
    y[i] = (L[i]/R)*(180/pi)

In [65]:
op_lon = np.zeros(A)
op_lat = np.zeros(A)
op_x = np.zeros(A)
op_y = np.zeros(A)

In [66]:
c = 0
i = 0
j = 1

while i < A:
    while j <= 17:
        op_lon[i] = lon[c]
        i = i+1
        j = j+1
    j = 1
    c = c+1

In [67]:
c = 0
i = 0
j = 1

while i < A:
    while j <= 17:
        op_lat[i] = lat[c]
        i = i+1
        j = j+1
    j = 1
    c = c+1

In [68]:
c = 0
i = 0
j = 1

while i < A:
    while j <= 17:
        op_x[i] = x[c]*cos((j-1)*(pi/8))
        i = i+1
        j = j+1
    j = 1
    c = c+1

In [69]:
c = 0
i = 0
j = 1

while i < A:
    while j <= 17:
        op_y[i] = y[c]*sin((j-1)*(pi/8))
        i = i+1
        j = j+1
    j = 1
    c = c+1

In [70]:
op = np.zeros((A, 2))

In [71]:
for i in range(A):
    op[i][0] = op_lon[i]+op_x[i]
    op[i][1] = op_lat[i]+op_y[i]

In [72]:
# Origin Polygons
op = pd.DataFrame(op, index=multi_index)
op.columns = ['lon', 'lat']

In [73]:
op.tail(4)

lon       lat
center arc                      
3000   14   225.633608 -2.087904
       15   226.011387 -1.835520
       16   226.263810 -1.457801
       17   226.352450 -1.012250

__Terminations__

In [74]:
lon = np.asarray(tc.lon)
lat = np.asarray(tc.lat)
L = np.asarray(tc.L)

In [75]:
theta = np.zeros(C)
r = np.zeros(C)
x = np.zeros(C)
y = np.zeros(C)

In [76]:
for i in range(C):
    theta[i] = lat[i]*(pi/180)
    r[i] = R*cos(theta[i])
    x[i] = (L[i]/r[i])*(180/pi)
    y[i] = (L[i]/R)*(180/pi)

In [77]:
tp_lon = np.zeros(A)
tp_lat = np.zeros(A)
tp_x = np.zeros(A)
tp_y = np.zeros(A)

In [78]:
c = 0
i = 0
j = 1

while i < A:
    while j <= 17:
        tp_lon[i] = lon[c]
        i = i+1
        j = j+1
    j = 1
    c = c+1

In [79]:
c = 0
i = 0
j = 1

while i < A:
    while j <= 17:
        tp_lat[i] = lat[c]
        i = i+1
        j = j+1
    j = 1
    c = c+1

In [80]:
c = 0
i = 0
j = 1

while i < A:
    while j <= 17:
        tp_x[i] = x[c]*cos((j-1)*(pi/8))
        i = i+1
        j = j+1
    j = 1
    c = c+1

In [81]:
c = 0
i = 0
j = 1

while i < A:
    while j <= 17:
        tp_y[i] = y[c]*sin((j-1)*(pi/8))
        i = i+1
        j = j+1
    j = 1
    c = c+1

In [82]:
tp = np.zeros((A, 2))

In [83]:
for i in range(A):
    tp[i][0] = tp_lon[i]+tp_x[i]
    tp[i][1] = tp_lat[i]+tp_y[i]

In [84]:
# Termination Polygons
tp = pd.DataFrame(tp, index=multi_index)
tp.columns = ['lon', 'lat']

In [85]:
tp.tail(4)

lon       lat
center arc                      
3000   14   214.996314 -1.705446
       15   215.454375 -1.399387
       16   215.760442 -0.941337
       17   215.867918 -0.401030

### ID ###

In [86]:
eddy_id = pd.DataFrame(index)
eddy_id = pd.concat([index, index], axis=1)
eddy_id.columns = ['Eddy ID', 'eddy id']
eddy_id = eddy_id.set_index('Eddy ID')

In [87]:
eddy_id.tail(4)

,eddy id
Eddy ID,
2997,2997
2998,2998
2999,2999
3000,3000


### Date ###

In [88]:
ds.tail(4)

,A,track,lon,j1,n,cyc,U,lat,L
Nobs,,,,,,,,,
41316,4.50767,3000,585.187988,2449043,1,-1,43.786400,-1.012250,129.462006
41317,5.60816,3000,577.690979,2449050,2,-1,91.454697,-0.200525,107.625999
41318,3.18113,3000,577.781982,2449057,3,-1,42.783001,-0.156585,190.341003
41319,2.38252,3000,574.455994,2449064,4,-1,23.117800,-0.401030,156.994995


In [89]:
j1 = np.asarray(ds.j1)

In [90]:
j1_o = np.zeros(M)
j1_t = np.zeros(M)

In [91]:
c = 0
i = 0

while i < N:
    c = int(c) + 1
    j1_o[c-1] = j1[i]
    i = i + int(point[c-1])
    j1_t[c-1] = j1[i-1]

In [92]:
j1_o = pd.DataFrame(j1_o)
j1_t = pd.DataFrame(j1_t)

In [93]:
# Start Julian Dates
sj = pd.concat([index, j1_o], axis=1)
sj.columns = ['Eddy ID', 'j1']
sj = sj.set_index('Eddy ID')

In [94]:
sj.tail(4)

,j1
Eddy ID,
2997,2449043.0
2998,2449043.0
2999,2449043.0
3000,2449043.0


In [95]:
# End Julian Dates
ej = pd.concat([index, j1_t], axis=1)
ej.columns = ['Eddy ID', 'j1']
ej = ej.set_index('Eddy ID')

In [96]:
ej.tail(4)

,j1
Eddy ID,
2997,2449064.0
2998,2449064.0
2999,2449064.0
3000,2449064.0


In [97]:
# Start Dates
sd = sj.j1.apply(jday_to_datetime)
sd = pd.DataFrame(sd)
sd = sd.rename(columns = {'j1': 'date'})

In [98]:
sd.tail(4)

,date
Eddy ID,
2997,1993-02-24 12:00:00
2998,1993-02-24 12:00:00
2999,1993-02-24 12:00:00
3000,1993-02-24 12:00:00


In [99]:
# End Dates
ed = ej.j1.apply(jday_to_datetime)
ed = pd.DataFrame(ed)
ed = ed.rename(columns = {'j1': 'date'})

In [100]:
ed.tail(4)

,date
Eddy ID,
2997,1993-03-17 12:00:00
2998,1993-03-17 12:00:00
2999,1993-03-17 12:00:00
3000,1993-03-17 12:00:00


In [101]:
# Durations in Days
dd = ed-sd

In [102]:
dd.date.max()

Timedelta('1596 days 00:00:00')

In [103]:
dd.tail(4)

,date
Eddy ID,
2997,21 days
2998,21 days
2999,21 days
3000,21 days


In [104]:
# Durations in Integers
di = np.zeros(M)

In [105]:
for i in range(M):
    di[i] = np.timedelta64(dd.date[i+1], 'D')/np.timedelta64(1, 'D')

In [106]:
di = pd.DataFrame(di)

In [107]:
di = pd.concat([index, di], axis=1)
di.columns = ['Eddy ID', 'day']
di = di.set_index('Eddy ID')
di.day = di.day.astype(int)

In [108]:
di.tail(4)

,day
Eddy ID,
2997,21
2998,21
2999,21
3000,21


### Area ###

In [109]:
radius = np.asarray(ds.L)

In [110]:
area = 2*pi*(radius**2)

In [111]:
aa = np.zeros(M)

In [112]:
i = 0
c = 0

while i < N:
    c = int(c) + 1
    aa[c-1] = area[i]
    i = i + int(point[c-1])

In [113]:
aa = pd.DataFrame(aa)

In [114]:
# Area in Square Kilometers
aa = pd.concat([index, aa], axis=1)
aa.columns = ['Eddy ID', 'area']
aa = aa.set_index('Eddy ID')

# Area in Square Meters
aa = aa*10**6

In [115]:
aa.tail(4)

,area
Eddy ID,
2997,3.507557e+10
2998,6.621897e+10
2999,3.622820e+10
3000,1.053088e+11


### Vorticity ###

In [116]:
from tqdm import tqdm

In [117]:
# Relative Vorticity
zeta = np.asarray((ds.U*0.01)/(ds.L*1000))

In [118]:
# Multi-Index Construction
arrays = [track, n]
tuples = list(zip(*arrays))
multi_index = pd.MultiIndex.from_tuples(tuples, names=['Eddy ID', 'Point ID'])

In [119]:
zeta = pd.DataFrame(zeta, index=multi_index)
zeta.columns = ['relative vorticity']

In [120]:
zeta.tail(4)

relative vorticity
Eddy ID Point ID                    
3000    1                   0.000003
        2                   0.000008
        3                   0.000002
        4                   0.000001

In [121]:
lav = np.zeros(M)

In [122]:
for i in tqdm(range(0, M)):
    lav[i] = zeta.loc[i+1].mean()

100%|██████████| 3000/3000 [00:04<00:00, 727.20it/s]


In [123]:
lav = pd.DataFrame(lav)

In [124]:
lav = pd.concat([index, lav], axis=1)
lav.columns = ['Eddy ID', 'zeta']
lav = lav.set_index('Eddy ID')

In [125]:
lav.tail(4)

,zeta
Eddy ID,
2997,1.215096e-06
2998,3.599635e-06
2999,8.633401e-07
3000,3.899964e-06


### GeoJSON ###

In [126]:
from geojson import LineString, Point, Polygon
from pymongo import MongoClient

In [127]:
client = MongoClient()
db = client.eddies

In [128]:
# Print Database Names
client.database_names()

[u'eddies', u'local']

In [129]:
# Print Collection Names
db.collection_names()

[u'ssh_eddies']

In [130]:
# Removes Documents from the Collection
result = db.ssh_eddies.remove()

/Applications/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:2: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
  from ipykernel import kernelapp as app


In [131]:
# Inserts Documents into the Collection
for i in tqdm(range(1, M+1)):

    # GeoJSON
    eddy = {
        '_id': int(eddy_id.loc[i]),
        'type': 'FeatureSet',
        'loc_start': [sc.loc[i]['lon'], sc.loc[i]['lat']],
        'loc_end': [ec.loc[i]['lon'], ec.loc[i]['lat']],
        'date_start': sd.loc[i]['date'],
        'date_end': ed.loc[i]['date'],
        'duration': di.loc[i]['day'],
        'area': aa.loc[i]['area'],
        'lav': lav.loc[i]['zeta'],
        'features': [
            {
                'type': 'Feature',
                'properties': {'name': 'start_center'},
                'geometry': Point(tuple(sc.loc[i][['lon', 'lat']].values))
            },
            {
                'type': 'Feature',
                'properties': {'name': 'end_center'},
                'geometry': Point(tuple(ec.loc[i][['lon', 'lat']].values))
            },
            {
                'type': 'Feature',
                'properties': {'name': 'trajectory'},
                'geometry': LineString([tuple(x) for x in ls.loc[i][['lon', 'lat']].values])
            },
            {
                'type': 'Feature',
                'properties': {'name': 'start_polygon'},
                'geometry': Polygon([[tuple(x) for x in op.loc[i][['lon', 'lat']].values]])
            },
            {
                'type': 'Feature',
                'properties': {'name': 'end_polygon'},
                'geometry': Polygon([[tuple(x) for x in tp.loc[i][['lon', 'lat']].values]])
            }
        ]    
    }
    
    # MongoDB
    result = db.ssh_eddies.insert_one(dict(eddy))

100%|██████████| 3000/3000 [00:40<00:00, 73.51it/s]
